In [1]:
import pandas as pd
import json
import seaborn as sns
from urllib.parse import urlparse
import numpy as np
import re

In [2]:
%cd ~/transuasion

/home/someshs/transuasion


In [3]:
# !wget https://github.com/wordnik/wordlist/raw/main/wordlist-20210729.txt

In [4]:
en = set(open('wordlist-20210729.txt').read().split())

In [5]:
df_pairs = pd.read_parquet('parallel_tweet_it0.parquet')
usernames = json.load(open('approved_usernames.json'))

FileNotFoundError: [Errno 2] No such file or directory: 'parallel_tweet_it0.parquet'

In [429]:
df_pairs = df_pairs[(df_pairs['character_difference']>5) & (df_pairs['date_diff']<=45) & (df_pairs['similarity']>=0.5) & ((df_pairs['media_x']!='') | (df_pairs['media_y']!='')) & (df_pairs['username'].isin(usernames))]

In [320]:
url = df_pairs['url'].drop_duplicates()
rurl = url.apply(lambda x: urlparse(x[x.find('next=')+5:]) if 'next=' in x else urlparse(x))
url_df = pd.DataFrame({'url':rurl, 'rurl':url})

In [321]:
url_df['netloc'] = url_df['url'].apply(lambda x: x.netloc)
url_df['path'] = url_df['url'].apply(lambda x: x.path)
url_df['query'] = url_df['url'].apply(lambda x: x.query)
url_df['fragment'] = url_df['url'].apply(lambda x: x.fragment)

In [322]:
url_df['netloc'].value_counts().head(30)

netloc
bit.ly                     57454
www.youtube.com            15124
buff.ly                     9741
twitter.com                 7842
www.liverpoolecho.co.uk     4870
freebeacon.com              3938
americanindependent.com     3901
ibm.co                      3250
www.facebook.com            2687
totalfratmove.com           1996
news.harvard.edu            1941
netflixlife.com             1852
www.totalrl.com             1640
on.fb.me                    1531
t.co                        1442
www.instagram.com           1363
houseandhome.com            1295
tsm.social                  1243
brook.gs                    1196
totalsororitymove.com       1132
cards.twitter.com           1112
uni.cf                      1095
on.natgeo.com               1085
mhpbooks.com                1040
po.st                       1018
soundcloud.com               939
today.ttu.edu                910
metro.co.uk                  814
sie.ag                       790
dailyorange.com              785
Nam

In [323]:
import re

def convert_url_to_tags(url):
    # Extract words from the URL using regular expressions
    words = re.findall(r'[A-Za-z0-9]+', url)

    # Process each word based on the specified conditions
    processed_words = []
    for word in words:
        # Break camel case
        words_split = re.findall(r'[a-z0-9]+|[A-Z][a-z0-9]*', word)
        processed_words.extend(words_split)

    # Filter words based on conditions
    filtered_words = [word for word in processed_words if (word.isalpha() and len(word)>1) or (word.isdigit() and int(word) <= 2025)]

    # Join the filtered words to create the desired string format
    result_string = ' '.join(filtered_words)

    return result_string

In [324]:
url_df["verb"] = url_df['path'].apply(convert_url_to_tags)
x = ' '.join(url_df["verb"]).lower().split()
remove_tags = {k for k,v in Counter(x).items() if v<3 and k} - en

def convert_url_to_tags(url):
    # Extract words from the URL using regular expressions
    words = re.findall(r'[A-Za-z0-9]+', url)

    # Process each word based on the specified conditions
    processed_words = []
    for word in words:
        # Break camel case
        words_split = re.findall(r'[a-z0-9]+|[A-Z][a-z0-9]*', word)
        processed_words.extend(words_split)

    # Filter words based on conditions
    filtered_words = [word for word in processed_words if (word.isalpha() and len(word)>1 and (not (word in remove_tags))) or (word.isdigit() and int(word) <= 2025)]

    # Join the filtered words to create the desired string format
    result_string = "', '".join(filtered_words)
    if len(result_string)>0:
        return " The webpage can be described by keywords: '" + result_string + "' ."
    else:
        return ""
    
def get_domain(d):
    d = [_d for _d in d.split('.') if not _d in ["www", "com"]]
    if 'ly' in d or "bitly" in d:
        return ""
    else:
        return " Its domain is " + '.'.join(d) + ' '
    
domain_verb = url_df['netloc'].apply(get_domain)
url_df['path'] = url_df['path'].apply(lambda x: '' if len(x)==7 else x)
url_df['verb'] = domain_verb + url_df['path'].apply(convert_url_to_tags)
url_df['verb']

In [378]:
url_df['verb'] = url_df['verb'].apply(lambda x: "The tweet has a webpage linked to it. " +x if len(x)>10 else x)
url_df['url'] = url_df['url'].astype(str)
url_verb = url_df.set_index('rurl')["verb"].to_dict()
json.dump(url_verb, open('url_verb.json', 'w'), indent=4)

In [1]:
def clean_tweet(tweet):
    masked_tweet = re.sub(r'@(\w+)', '<USERNAME>', tweet)
    usernames = re.findall(r'(@\w+)', tweet)
    links = re.findall(r'https?://\S+|www\.\S+', tweet)
    masked_tweet = re.sub(r'https?://\S+|www\.\S+', '<HYPERLINK>', masked_tweet)

    return {
        'cleaned_tweet': masked_tweet,
        'usernames': ' '.join(usernames),
    }

# Pair Simulation

In [401]:
PREFIX = """{} wrote a tweet.
{} Compare the engagement levels of two tweets: Tweet (A) and Tweet (B).
Determine which tweet will get a higher number of likes, answer (A) or (B).
Provide the ratio of likes for (A) and (B)
"""

INSTRUCTION = """Tweet (A) with masked mentions {}:
'''{}'''

Tweet (B) with masked mentions {}:
'''{}'''

"""

ANSWER = """{} will be liked {} times more"""

In [270]:
pairs_0 = df_pairs[(df_pairs['likes_x'] * df_pairs['likes_y']) == 0]
pairs_n0 = df_pairs[(df_pairs['likes_x'] * df_pairs['likes_y']) != 0]

pairs_00 = pairs_0[pairs_0['likes_x'] + pairs_0['likes_y']==0] #both zero
#pairs_0n0 = pairs_0[pairs_0['likes_x'] + pairs_0['likes_y']>0] #single zero

pairs_nen = pairs_n0[pairs_n0['likes_x'] == pairs_n0['likes_y']] #non zero equals
pairs_nnen = pairs_n0[pairs_n0['likes_x'] != pairs_n0['likes_y']] #non zero unequals

pairs_nnen_close = pairs_nnen[(pairs_nnen['likes_x']+pairs_nnen['likes_y']<10) & (pairs_nnen['likes_ratio']<2)] #unequal non zero-close small values
pairs_nnen_remain = pairs_nnen[~((pairs_nnen['likes_x']+pairs_nnen['likes_y']<10) & (pairs_nnen['likes_ratio']<2))] #unequal non zero large/far values

pairs_nen_close = pairs_nen[(pairs_nen['likes_x']+pairs_nen['likes_y']<10)] #equal non zero small values
pairs_nen_remain = pairs_nen[~((pairs_nen['likes_x']+pairs_nen['likes_y']<10))] #equal non zero large values

In [271]:
pairs_nen_remain['tag']="High Non-Zero Equal Pairs"
pairs_nen_close['tag']="Low Non-Zero Equal Pairs"
pairs_nnen_close['tag']="Low Non-Zero Un-Equal Pairs"
pairs_nnen_remain['tag']="High Non-Zero Un-Equal Pairs"
pairs_00['tag']="Zero Pairs"

<ipython-input-271-a2e61a1d4ba8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_nen_remain['tag']="High Non-Zero Equal Pairs"
<ipython-input-271-a2e61a1d4ba8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_nen_close['tag']="Low Non-Zero Equal Pairs"
<ipython-input-271-a2e61a1d4ba8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [272]:
pd.concat([pairs_nen_remain, pairs_nen_close, pairs_nnen_close, pairs_nnen_remain, pairs_00])['tag'].value_counts()

tag
Zero Pairs                      623666
High Non-Zero Un-Equal Pairs    521620
Low Non-Zero Un-Equal Pairs     224153
Low Non-Zero Equal Pairs        173191
High Non-Zero Equal Pairs        17622
Name: count, dtype: int64

In [306]:
pair_sim = pd.concat([
    pairs_nen_remain,
    pairs_nnen_remain,
    pairs_nen_close.sample(frac=0.3),
    pairs_nnen_close.sample(frac=0.3),
    pairs_00.sample(frac=0.1)
]).sample(frac=1)

In [307]:
r1 = (pair_sim['likes_x']+0.11) / (pair_sim['likes_y']+0.11).values
r2 = (pair_sim['likes_y']+0.11) / (pair_sim['likes_x']+0.11).values
pair_sim['likes_ratio'] = np.max(np.array([r1,r2]), axis=0)

tag
High Non-Zero Un-Equal Pairs    521620
Low Non-Zero Un-Equal Pairs      67246
Zero Pairs                       62367
Low Non-Zero Equal Pairs         51957
High Non-Zero Equal Pairs        17622
Name: count, dtype: int64

In [402]:
pair_sim['masked_tweet_x'] = pair_sim['tweet_x'].apply(clean_tweet)
pair_sim['masked_tweet_y'] = pair_sim['tweet_y'].apply(clean_tweet)

In [416]:
print(.tolist()[-2])

IBMWatson wrote a tweet.
The tweet has a webpage linked to it.  Its domain is techrepublic  The webpage can be described by keywords: 'article', 'the', 'future', 'of', 'ai', '10', 'scenarios', 'ibm', 'is', 'already', 'working', 'on' . Compare the engagement levels of two tweets: Tweet (A) and Tweet (B).
Determine which tweet will get a higher number of likes, answer (A) or (B).
Provide the ratio of likes for (A) and (B)

Tweet (A) with masked mentions @TechRepublic:
'''10 practical scenarios with how #AI is used to assist human problem-solving: <HYPERLINK> via <USERNAME>'''

Tweet (B) with masked mentions @TechRepublic:
'''How is #AI being used to assist human problem-solving? Here are 10 practical scenarios: <HYPERLINK> via <USERNAME>'''




In [408]:
pair_sim.apply(lambda x: ANSWER.format('(A)' if x.likes_x > x.likes_y else '(B)', np.round(x.likes_ratio, 2)),axis=1)

31461922    (A) will be liked 1.14 times more
5058370     (A) will be liked 2.93 times more
25773527    (A) will be liked 1.29 times more
3837334     (B) will be liked 1.88 times more
36439930    (B) will be liked 1.49 times more
dtype: object

In [417]:
rpair_sim = pair_sim[['id_x', 'id_y', 'url', 'likes_ratio']]
rpair_sim['instruction'] = pair_sim.apply(lambda x: PREFIX.format(x.username, url_verb.get(x.url, '')) + '\n' + INSTRUCTION.format(x.masked_tweet_x['usernames'], x.masked_tweet_x['cleaned_tweet'], x.masked_tweet_y['usernames'], x.masked_tweet_y['cleaned_tweet']), axis=1)
rpair_sim['answer'] = pair_sim.apply(lambda x: ANSWER.format('(A)' if x.likes_x > x.likes_y else '(B)', np.round(x.likes_ratio, 2)),axis=1)

In [430]:
pair_ids = set(rpair_sim['id_x']).union(set(rpair_sim['id_y']))

In [431]:
len(pair_ids)

544067

# Transuasion